In [1]:
import pandas as pd
import numpy as np

In [2]:
class MultipleLinearRegression:
    def __init__(self):
        self.coefficients = None
        #constructor to the class which set coefficients to none
    def fit(self, X, y):
        # Add a column of ones to the design matrix X
        X_design = [[1] + row for row in X] #adding ones column to calculate b0 value
        n = 0
        for _ in X[0]:
           n += 1


        m = 0
        for row in  y:
         m += 1   
        #n = len(X_design[0])#finding the no.of columns
        #m = len(y)#finding no.of rows of dependent variable
        
        # Initialize the normal equations matrix A and the right-hand side vector b
        A = [[0] * n for i in range(n)]#an empty array with shape as n*n(2D)
        b = [0] * n #1D array
        
        # Fill in the elements of A and b
        for i in range(n):
            for j in range(n):
                for k in range(m):
                    A[i][j] += X_design[k][i] * X_design[k][j]
            for k in range(m):
                b[i] += X_design[k][i] * y[k]
        
        # Solve the system of linear equations Ax = b using Gaussian elimination
        for i in range(n):
            for j in range(i + 1, n):
                factor = A[j][i] / A[i][i]
                for k in range(i + 1, n):
                    A[j][k] -= factor * A[i][k]
                b[j] -= factor * b[i]
                
        self.coefficients = [0] * n
        for i in range(n - 1, -1, -1):
            for j in range(i + 1, n):
                b[i] -= A[i][j] * self.coefficients[j]
            self.coefficients[i] = b[i] / A[i][i]

    def predict(self, X):
        X_design = [[1] + row for row in X]
        n_rows = 0
        for row in X_design:
          n_rows += 1

        n_cols = 0
        for col in self.coefficients:
          n_cols += 1  

        y_pred = [0] * n_rows
        for i in range(n_rows):
          y_pred[i] = 0
          for j in range(n_cols):
            y_pred[i] += X_design[i][j] * self.coefficients[j]
  

        return y_pred




In [3]:
df=pd.read_csv("/content/insurance.csv")
df
#y = df[["price"]]
#x =df.drop(["price","date","id"],axis=1)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [4]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [5]:
df["smoker"].value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [6]:
#encoding categorical i.e region,smoker,sex- smoker and sex we can use binary encoding, region -label or ordinal encoding
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df[['sex','smoker','region']]).toarray())

#merge one-hot encoded columns back with original DataFrame
final_df = df.join(encoder_df)

In [7]:
#from sklearn import preprocessing#
#label_encoder = preprocessing.LabelEncoder()
#df['sex']= label_encoder.fit_transform(df['sex'])
#df['smoker']= label_encoder.fit_transform(df['smoker'])



In [8]:
final_df=final_df.drop(['sex','smoker','region'],axis=1)

In [9]:
y=final_df[["charges"]]
x=final_df.drop(["charges"],axis=1)

In [11]:
ratio = 0.75
 
total_rows = df.shape[0]
train_size = int(total_rows*ratio)
 
# Split data into test and train
x_train =(x[0:train_size].values)

y_train=(y[0:train_size].values)
x_test =(x[train_size:].values)

y_test =(y[train_size:].values)

In [12]:
mlr = MultipleLinearRegression()

# Fit the model to the data
mlr.fit(x_train, y_train)

# Make predictions on the test data
y_pred = mlr.predict(x_test)

In [13]:
x_train.dtype

dtype('float64')

In [14]:
# Calculate the sum of squared residuals
residuals = y_pred - y_test
ss_residuals = np.sum(residuals**2)

# Calculate the total sum of squares
mean_y = np.mean(y_test)
ss_total = np.sum((y_test - mean_y)**2)

# Calculate R^2
R2 = 1 - (ss_residuals / ss_total)
print(R2)

0.7321359094584851
